In [1]:
import pandas as pd

diabetes = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")

In [2]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, test_size = 0.3, stratify = diabetes.Diabetes_binary, random_state = 1234)
test, validation = train_test_split(test, test_size = 0.3, stratify = test.Diabetes_binary, random_state = 1234)

In [3]:
len(train), len(test), len(validation)

(177576, 53272, 22832)